In [1]:
import pickle

# Load the pickle file
with open('Vectordatabase/financial_chunks_metadata_sentences.pkl', 'rb') as file:
    text_chunks = pickle.load(file)

In [2]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Initialize SentenceTransformer model
text_embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Load the existing FAISS index
index = faiss.read_index('Vectordatabase/financial_docs_text_index_sentences.faiss')

def retrieve_similar(query, k=1):
    # Get query embedding
    query_embedding = text_embedder.encode([query])
    # Search in FAISS index
    distances, indices = index.search(query_embedding.astype(np.float32), k)
    return indices, distances

# Example query
query = "what food can I get to eat"
indices, distances = retrieve_similar(query)

# Display results
print("Query:", query)
print("Retrieved documents:")
for i, idx in enumerate(indices[0]):
    print(f"{i + 1}: Document Index {idx} (Distance: {distances[0][i]:.4f})")
    print(text_chunks[idx]['raw_text'])

c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Query: what food can I get to eat
Retrieved documents:
1: Document Index 17 (Distance: 1.1764)
Our FoodOur Menu. Our menu features unrivaled Chicago-style street food and all-American favorites —s omething for everyone! We're known for
our famous Chicago-style hot dogs, Italian beef sandwiches, chopped salad, cheese fries, homemade chocolate cake, and chocolate cake
shake. Our Italian beef is slow-roasted for four hours, thinly sliced, and served on freshly-baked French bread. Then, it's dipped in hot
gravy made with our homemade blend of seasonings that we've been perfecting for 60 years.


In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2")
prompt = "Tell me a joke about llama"
result = llm.invoke(prompt)
print(result)

Here's one:

Why did the llama refuse to play poker?

Because he always got fleeced! (get it?)


In [ ]:
def generate_response(query):
    indices, distances = retrieve_similar(query)
    context = "\n".join([text_chunks[idx]['raw_text'] for _, idx in enumerate(indices[0])])
    # Prepare the input for the T5 model
    input_text = f"question: {query} context: {context}. Let me know if the context was useful."



    response = llm.invoke(input_text)

    return context, response

In [38]:
# Example query
query = "What type of customers do Portillos cater to?"
context, response = generate_response(query)

print("Query:", query)
print("Context:", context)
print("Generated Response:", response)

Query: What type of customers do Portillos cater to?
Context: Overview of Portillo's
Portillo’s serves iconic Chicago street food in high-energy, multichannel restaurants designed to ignite the senses and create memorable
dining experiences. Since our founding in 1963 in a small trailer that Dick Portillo called “The Dog House,” we have grown to become a
treasured brand with a passionate (some might say obsessed) nationwide following. Our diverse menu features all-American favorites
such as Chicago-style hot dogs and sausages, Italian beef sandwiches, char-grilled burgers, chopped salads, crinkle-cut fries, homemade
chocolate cake and signature chocolate cake shake. We create a consumer experience like no other by combining the best attributes of
fast-casual and quick-service concepts with an exciting energy-filled atmosphere in a restaurant model capable of generating tremendous
volumes. Nearly all of our restaurants were built with double lane drive-thrus and have been thoughtfully d

In [26]:
llm = Ollama(model="llama3.2:1b")

# Example query
query = "What type of customers do Portillos cater to?"
context, response = generate_response(query)

print("Query:", query)
print("Context:", context)
print("Generated Response:", response)

Query: What type of customers do Portillos cater to?
Context: engages senses operating model focused getting delicious madetoorder food guests quickly atmosphere makes experience delicious meal guests walk portillo ’ s get experience completely different typical chain restaurant visit restaurants engage senses create fun relaxed memorable occasion dining areas evoke nostalgia local influences two portillo ’ s alike restaurants themed decor ranging 1930s prohibition motif 1950s diner 1960s
Generated Response: Your context is very informative and helpful in understanding what type of customers Portillos cater to. Here's a breakdown of your context:

- "Portillo's" refers to a specific fast-food restaurant chain that serves Italian beef sandwiches, hot dogs, and other American-style comfort food.
- The operating model focused on getting delicious, made-to-order food guests quickly suggests that the primary goal is to provide speedy service with high-quality food to keep customers satisfie

In [28]:
def generate_contextless_response(query):


    response = llm.invoke(query)

    return response

# Example query
query = "What type of customers do Portillos cater to?"
response = generate_contextless_response(query)

print("Query:", query)
print("Generated Response:", response)

Query: What type of customers do Portillos cater to?
Generated Response: Portillo's is a popular American-style hot dog and Italian beef restaurant chain, particularly known for their hot dogs. Based on their menu and target market, it appears that Portillo's primarily caters to the following types of customers:

1. Foodies: People who enjoy trying new flavors and cuisines, especially those from different cultural backgrounds.
2. Hot Dog Enthusiasts: Individuals who are passionate about hot dogs and look for high-quality, authentic Chicago-style hot dogs.
3. Italian Beef Lovers: Fans of Italian beef sandwiches, which Portillo's is famous for serving, as well as other Italian-American dishes.
4. Students and Young Professionals: People working in the surrounding areas, particularly those living in or near the city of Chicago, who might appreciate the casual dining atmosphere and affordable prices.

Portillo's also tends to attract a more affluent demographic due to their upscale restaur

In [29]:
# Example query
query = "What type of customers do Portillos cater to? You not have to use the context provided to help with your answer. Rate how useful the context was."
context, response = generate_response(query)

print("Query:", query)
print("Context:", context)
print("Generated Response:", response)

Query: What type of customers do Portillos cater to? You not have to use the context provided to help with your answer. Rate how useful the context was.
Context: engages senses operating model focused getting delicious madetoorder food guests quickly atmosphere makes experience delicious meal guests walk portillo ’ s get experience completely different typical chain restaurant visit restaurants engage senses create fun relaxed memorable occasion dining areas evoke nostalgia local influences two portillo ’ s alike restaurants themed decor ranging 1930s prohibition motif 1950s diner 1960s
Generated Response: I would rate the usefulness of the context as 8 out of 10.

The context provides valuable information about Portillos, such as their unique atmosphere, menu offerings, and dining experience. It also highlights the differences between two of their locations, which can help readers understand how different settings can affect the customer experience. The mention of local influences, no

In [31]:
# Example query
query = "Is Portillo's scalable? If the context doesn't help, do not use it. Rate how useful the context was."
context, response = generate_response(query)

print("Query:", query)
print("Context:", context)
print("Generated Response:", response)

Query: Is Portillo's scalable? If the context doesn't help, do not use it. Rate how useful the context was.
Context: december 6 2019 among portillo ’ s holdings llc borrower phd intermediate llc ubs ag stamford branch administrative agent 107 collateral agent lenders time time party thereto incorporated reference company ’ s registration statement form s1 filed september 27 2021 portillo ’ s inc 2021 equity incentive plan incorporated reference companys form 10q filed 108
Generated Response: I would rate the usefulness of the context as neutral to somewhat negative.

The context is quite long and provides a lot of technical details, which can be overwhelming without prior knowledge or interest in Portillo's Holdings LLC (PHL). While it's helpful for someone who wants to dive deep into the company's financials, it may not provide much value to an outside observer. The mention of specific documents like S1 filing and 10Q filing also adds a layer of complexity that might make it difficult

In [32]:
# Example query
query = "Is Portillo's scalable?"
response = generate_contextless_response(query)

print("Query:", query)
print("Generated Response:", response)

Query: Is Portillo's scalable?
Generated Response: Portillo's is a popular fast-casual restaurant chain that serves Italian and Chicago-style beef hot dogs. While it has grown significantly in popularity, its scalability can be debated.

Here are some factors to consider:

Pros:

1. **Franchise model**: Portillo's has a franchise model, which allows existing restaurants to expand their operations by buying into the brand. This makes it easier for new owners to enter the market.
2. **Strong brand recognition**: Portillo's has built a loyal following over the years, which can help attract customers from across the region.

Cons:

1. **Limited menu offerings**: Portillo's is known for its hot dogs and Italian-American cuisine, but its menu offerings may not be as diverse as those of larger restaurants.
2. **Supply chain complexity**: As the brand expands, managing a large number of locations can become increasingly complex, particularly when it comes to food quality control and logistics.

In [33]:
# Example query
query = "Are they scalable? Company: Portillo's"
response = generate_contextless_response(query)

print("Query:", query)
print("Generated Response:", response)

Query: Are they scalable? Company: Portillo's
Generated Response: Portillo's is a Chicago-based fast-casual restaurant chain, and as such, their scalability is a significant aspect to consider. Here are some insights:

**Strengths:**

1. **Existing infrastructure**: Portillo's has an established presence in the Midwest region, with multiple locations across Illinois, Indiana, and Wisconsin. This existing infrastructure provides a solid foundation for scaling.
2. **Brand recognition**: The Portillo's brand is well-known and trusted in the Midwest, which can help attract new customers and increase customer loyalty as the company scales.
3. **Menu offerings**: The menu at Portillo's has evolved over time to include more options and limited-time flavors, making it easier to expand and adapt to changing consumer preferences.

**Challenges:**

1. **Supply chain management**: With multiple locations, managing supply chains for ingredients, meat, and produce can become complex and costly.
2. *

In [40]:
# Example query
query = "Is Portillo's scalable?"
context, response = generate_response(query)

print("Query:", query)
print("Context:", context)
print("Generated Response:", response)

Query: Is Portillo's scalable?
Context: Overview of Portillo's
Portillo’s serves iconic Chicago street food in high-energy, multichannel restaurants designed to ignite the senses and create memorable
dining experiences. Since our founding in 1963 in a small trailer that Dick Portillo called “The Dog House,” we have grown to become a
treasured brand with a passionate (some might say obsessed) nationwide following. Our diverse menu features all-American favorites
such as Chicago-style hot dogs and sausages, Italian beef sandwiches, char-grilled burgers, chopped salads, crinkle-cut fries, homemade
chocolate cake and signature chocolate cake shake. We create a consumer experience like no other by combining the best attributes of
fast-casual and quick-service concepts with an exciting energy-filled atmosphere in a restaurant model capable of generating tremendous
volumes. Nearly all of our restaurants were built with double lane drive-thrus and have been thoughtfully designed with a layout 

In [11]:
llm = Ollama(model="llama3.2:1b")
def generate_response(query):
    indices, distances = retrieve_similar(query)
    context = "\n".join([text_chunks[idx]['raw_text'] for _, idx in enumerate(indices[0])])
    # Prepare the input for the T5 model
    input_text = f"question: {query} context: {context}."



    response = llm.invoke(input_text)

    return context, response

In [12]:
# Example query
query = "How does Portillo’s manage supply chain risks?"
context, response = generate_response(query)

print("Query:", query)
print("Context:", context)
print("Generated Response:", response)

Query: How does Portillo’s manage supply chain risks?
Context: Portillo's Inc. Form 10-K | 34Table of Contents
ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK
Commodity and Food Price Risks
We are exposed to commodity price risks. Many of the ingredients we use to prepare our food, as well as our packaging materials, are
commodities or are affected by the price of other commodities. Factors that affect the price of commodities are generally outside of our
control and include raw material inputs, geopolitical events, weather conditions, currency markets and global supply and demand
dynamics, among other items. In order to minimize the impact of fluctuations in price and availability, we monitor the primary commodities we purchase and may enter
into purchasing contracts and pricing arrangements when considered to be advantageous.
Generated Response: According to Portillo's Inc.'s Form 10-K, the company manages supply chain risks through various means, including:

1. M

In [ ]:
llm = Ollama(model="llama3.2")
query = "How does Portillo’s manage supply chain risks?"
context, response = generate_response(query)

print("Query:", query)
print("Context:", context)
print("Generated Response:", response)